In [ ]:
import os
from functools import partial
import yaml
import hashlib
from io import BytesIO
from zipfile import ZipFile

from IPython import display
import ipywidgets
from ipywidgets import Label, Dropdown, HBox, VBox, Text, Textarea, Button, Valid, Tab, HTML
import traitlets
from traitlets import Unicode
from tornado.httpclient import AsyncHTTPClient
import jinja2

import db
db.use_db()

## File Upload Widget

In [ ]:
class FileWidget(ipywidgets.DOMWidget):
    _view_name = Unicode('FilePickerView').tag(sync=True)
    _view_module = Unicode('filepicker').tag(sync=True)

    value = Unicode().tag(sync=True)
    filename = Unicode().tag(sync=True)

    def __init__(self, **kwargs):
        """Constructor"""
        ipywidgets.DOMWidget.__init__(self, **kwargs) # Call the base.

        # Allow the user to register error callbacks with the following signatures:
        #    callback()
        #    callback(sender)
        self.errors = ipywidgets.CallbackDispatcher(accepted_nargs=[0, 1])

        # Listen for custom msgs
        self.on_msg(self._handle_custom_msg)

    def _handle_custom_msg(self, content):
        """Handle a msg from the front-end.

        Parameters
        ----------
        content: dict
            Content of the msg."""
        if 'event' in content and content['event'] == 'error':
            self.errors()
            self.errors(self)

In [ ]:
%%javascript

requirejs.undef('filepicker');

define('filepicker', ["jupyter-js-widgets"], function(widgets) {

    var FilePickerView = widgets.DOMWidgetView.extend({

        render: function() {
            // Render the view.
            this.setElement($('<input />')
                .attr('type', 'file'));
        },

        events: {
            // List of events and their handlers.
            'change': 'handle_file_change',
        },

        handle_file_change: function(evt) {
            // Handle when the user has changed the file.

            // Retrieve the first (and only!) File from the FileList object
            var file = evt.target.files[0];
            if (! file) { 
                // The file couldn't be opened.  Send an error msg to the
                // back-end.
                this.send({ 'event': 'error' });
                return ;
            }
            
            // check if has 'yml' extension

            // Read the file's textual content and set value to those contents.
            var that = this;
            var file_reader = new FileReader();
            file_reader.onload = function(e) {
                that.model.set('value', e.target.result);
                that.touch();
                console.log('sent from client side')
            }
            file_reader.readAsText(file);

            // Set the filename of the file.
            this.model.set('filename', file.name);
            this.touch();
        },
    });

    return {
        FilePickerView: FilePickerView
    };
});

---
## Utilities

In [ ]:
def get_exam_names():
    with db.db_session:
        return list(db.orm.select(ex.name for ex in db.Exam))
    
def get_exam_data(exam_name):
    if not exam_name:
        return None
    with db.db_session:
        exam = db.Exam.get(name=exam_name)
        if not exam:
            return None
        yaml_path = exam.yaml_path
    try:
        return db.read_yaml(yaml_path)
    except Exception:
        return None

def observe(widget, **kwargs):
    return partial(widget.observe, **kwargs)

---
## Widget Init

In [ ]:
yaml_upload_content = Textarea()
yaml_upload_submit = Button(description='Upload metadata')
yaml_upload_valid = Valid()
yaml_upload_status = Label()
yaml_upload_box = VBox([yaml_upload_content,
                        HBox([yaml_upload_submit, yaml_upload_status, yaml_upload_valid])])

yaml_upload_valid.layout.display = 'none'
yaml_upload_valid.layout.margin = "0 10px"
yaml_upload_content.width = "100%"
yaml_upload_content.height = "300px"

# Register an event to echo the filename when it has been changed.
@yaml_upload_submit.on_click
def submit_yaml(change=None):
    if not yaml_upload_content.value:
        return
    # validate yaml
    yaml_upload_valid.layout.display = 'none'
    yaml_upload_status.value = "Validating metadata"
    try:
        yml = yaml.load(yaml_upload_content.value)
        
        try:
            yml = db.clean_yaml(yml)  # attempt to clean
        except Exception:
            pass  # if the YAML is not valid parsing will raise anyway
        version, exam_name, qr, widgets = db.parse_yaml(yml)
        if select_exam.value and select_exam.value != exam_name:
            raise ValueError('Exam in metadata file does not match selected exam')
        # If there is already yaml for this exam, load it now so we can
        # compute a diff later
        existing_exam_data = get_exam_data(exam_name)
        if existing_exam_data:
            existing_yml_version, *_, existing_widgets = existing_exam_data
            if not all(v == 1 for v in (version, existing_yml_version)):
                raise ValueError('Exam data for {} already exists, and updating it requires both the old '
                                 'and new YAML to be version 1'.format(exam_name))
            if not existing_widgets.shape == widgets.shape:
                raise ValueError('Exam data for {} already exists, and contains a different number of '
                                 'exam problems than {}'.format(exam_name, yaml_upload.filename))
    except ValueError as exc:
        yaml_upload_valid.value = False
        yaml_upload_valid.layout.display = 'inline'
        yaml_upload_status.value = str(exc)
        return
    except Exception as exc:
        yaml_upload_valid.value = False
        yaml_upload_valid.layout.display = 'inline'
        yaml_upload_status.value = "Invalid metadata"
        return

    # save yaml
    yaml_filename = exam_name + '.yml'
    yaml_upload_status.value = "Saving " + yaml_filename
    try:
        with open(yaml_filename, 'w') as f:
            f.write(yaml.dump(yml))
    except Exception:
        yaml_upload_valid.value = False
        yaml_upload_valid.layout.display = 'inline'
        yaml_upload_status.value = "Failed to save " + yaml_filename
        return
    
    yaml_upload_status.value = "Adding exam to database"
    
    try:
        if existing_exam_data:
            db.update_exam(exam_name, yaml_filename)
            yaml_upload_status.value = "Updated problem names for {}".format(exam_name)
        else:
            db.add_exam(yaml_filename)
            yaml_upload_status.value = "Added exam {} to database".format(exam_name)
        os.makedirs(exam_name + '_data', exist_ok=True)
    except Exception as exc:
        yaml_upload_valid.value = False
        yaml_upload_valid.layout.display = 'inline'
        yaml_upload_status.value = "Failed to add exam to database"
        return
    finally:
        # XXX: use of global variables
        # update list of exams
        select_exam.options = [None] + get_exam_names()
        select_exam.value = exam_name

    yaml_upload_status.value = "Metadata imported successfully"
    yaml_upload_valid.value = True
    yaml_upload_valid.layout.display = 'inline'

In [ ]:
pdf_label = Label("URL", margin="7px 10px")
pdf_url = Text(placeholder="https://dropbox.com/sh/af45ed301bd?dl=0")
pdf_import = Button(description="Import scanned PDF")

pdf_import_status = Label("To import a PDF, first select an exam")
pdf_import_valid = Valid()
pdf_import_valid.layout.display = 'none'
pdf_import_valid.layout.margin = "0 10px"

pdf_import_box = VBox([HBox([pdf_label ,pdf_url, pdf_import]),
                       HBox([pdf_import_status, pdf_import_valid])])
# initially no exam is selected, so we disable this
pdf_url.disabled = pdf_import.disabled = True

# this is global state to prevent the UI triggering another download
# while there is one in progress
pdf_download_client = None


def get_new_client():
    # We should probably use a streaming client, but for simplicity we just set the max buffer size to 500MB
    return AsyncHTTPClient(max_buffer_size=int(500E8))


def pdf_name(pdf_content):
    return hashlib.sha256(pdf_content).hexdigest()[:15] + '.pdf'


def handle_pdf(pdf, name='PDF'):
    exam_dir = select_exam.value + '_data'
    filename = os.path.join(exam_dir, pdf_name(pdf))
    pdf_import_status.value = 'Saving {}'.format(name)
    try:
        with open(filename, 'wb') as f:
            f.write(pdf)
    except Exception as e:
        raise RuntimeError('Failed to save {}'.format(name))

    pdf_import_status.value = 'Importing {}'.format(name)
    try:
        missing_qrs = db.process_pdf(filename, select_exam.value + '.yml')        
    except Exception as e:
        raise RuntimeError('Failed to import {}: {}'.format(name, str(e)))
    
    pdf_import_valid.layout.display = 'inline'

    if missing_qrs:
        pdf_import_valid.value = True
        pdf_import_status.value = ('Warning: could not process pages {}'
                                   .format(', '.join(str(i) for i in missing_qrs)))
    else:
        pdf_import_valid.value = True
        pdf_import_status.value = 'Imported {}'.format(name)

def handle_zip(zipcontents):
    exam_dir = select_exam.value + '_data'
    pdf_import_status.value = 'Extracting PDFs from ZIP archive'
    with ZipFile(BytesIO(zipcontents)) as zf:
        pdf_files = list(filter(lambda n: n.endswith('.pdf'), zf.namelist()))
        for pdf in pdf_files:
            pdf_import_status.value = 'Extracting {}'.format(pdf)
            handle_pdf(zf.open(pdf).read(), name=pdf)
            pdf_import_valid.layout.display = 'none'  # set at the end of handle_pdf
    
    pdf_import_valid.value = True
    pdf_import_valid.layout.display = 'inline'
    pdf_import_status.value = 'Imported: {}'.format(', '.join(pdf_files))        


content_type_router = {
    'application/pdf': handle_pdf,
    'application/zip': handle_zip,
}

def handle_response(response):    
    global rsp, pdf_download_client
    rsp = response
    try:
        if response.error:
            raise RuntimeError('Failed to fetch file(s)', response.error)
        try:
            handler = content_type_router[response.headers['Content-Type']]
            handler(response.body)
        except KeyError:
            raise RuntimeError('Link does not point to a PDF/ZIP file')
    except Exception as exc:
        pdf_import_valid.value = False
        pdf_import_valid.layout.display = 'inline'
        pdf_import_status.value = str(exc)
    finally:
        pdf_download_client = None  # allow next upload


def url_rewrite(url):
    """Correct common problems occurring in the download link."""
    if url.endswith('/'):
        url = url[:-1]
    if 'dropbox' in url[:20].lower() and url.endswith('dl=0'):
        return url[:-1] + '1'
    elif 'surfdrive.nl' in url[:20].lower() and not url.endswith('/download'):
        url = url + '/download'
    else:
        return url

@pdf_import.on_click
def _(sender):
    global pdf_download_client
    if pdf_download_client:
        return  # download already underway
    pdf_download_client = get_new_client()
    # 30 second timeout
    url = url_rewrite(pdf_url.value.strip())
    pdf_download_client.fetch(url, handle_response, request_timeout=30)
    pdf_import_valid.layout.display = 'none'
    pdf_import_status.value = 'Fetching file'


In [ ]:
exam_upload_summary = jinja2.Template('''
<h4> Complete Submissions uploaded: {{submissions | length}} </h4>
{% if partial_submissions %}
<h4> Partial submissions uploaded: {{partial_submissions | length}} </h4>
{% endif %}
''')


def render_summary(exam):
    if not exam:
        exam_upload_summary_html.value = "To see a summary first select an exam"
        return
    env = {}
    
    _, _, qr, _ = db.read_yaml(exam + '.yml')
    num_pages = len(qr)
    env['num_pages'] = num_pages
    
    with db.db_session:
        submissions = db.orm.select(s for s in db.Submission if s.exam.name == exam)
        
        submissions = [dict(copy_number=s.copy_number, pages=len(s.pages)) for s in submissions]
        partial_submissions = [s for s in submissions if s['pages'] < num_pages]
        
    env['submissions'] = submissions
    env['partial_submissions'] = partial_submissions

    exam_upload_summary_html.value = exam_upload_summary.render(**env)

exam_upload_summary_html = HTML()
render_summary(None)

In [ ]:
select_exam = Dropdown(description='Exam', options= [None] + get_exam_names())

@observe(select_exam, names="value")
def _(state):
    value = state['new']
    pdf_url.disabled = pdf_import.disabled = value is None
    pdf_import_status.value = "To import a PDF, first select an exam" if value is None else ''
    render_summary(value)
    yaml_upload_content.value = ''
    if value:
        with open(value+'.yml') as file:
            yaml_upload_content.value = file.read()

---
## UI Layout

In [ ]:
select_exam

In [ ]:
#content = Tab(children=[yaml_upload_box, exam_upload_summary_html, pdf_import_box,])
content = Tab(children=[yaml_upload_box, pdf_import_box,])
content.set_title(0, 'Upload exam metadata')
content.set_title(1, 'Import scanned PDFs')
#content.set_title(1, 'Exam summary')
#content.set_title(2, 'Import scanned PDFs')
content